In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Getting Headlines About Mars

In [2]:
# Set up spliner variables. Make it headless so it doesn't open a browser each time.
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Send it to the nasa site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [3]:
# Convert the browser's webpage to html and turn it into soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Find the first title and body
news_title = soup.find("div", class_="list_text").find("div", class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text

In [5]:
print(news_title)
print(news_p)

The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover
Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.


# Grabbing Nasa's Featured Mars Image

In [6]:
# Do it again for the images
url_base = "https://www.jpl.nasa.gov"
url = url_base+"/spaceimages/?search=&category=Mars"
browser.visit(url)

In [7]:
# Turn it into soup! 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
# The image is in a style tag, so let's grab that. 
bg_img = soup.find("article", class_="carousel_item")

In [9]:
# Now let's split it out of the style tag
# Convert it to a string, and split it on the ' around the url
image = (str(bg_img)).split("('", 1)[1].split("')")[0]

# Get a full url by combining it with the base url
featured_image_url = url_base+image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17932-1920x1200.jpg


# Getting the Weather on Mars from Twitter

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [11]:
# Turn it into soup! 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
tweet_article = soup.find("article", {"role" : "article", "tabindex":0})
spans = tweet_article.find_all("span")
for span in spans:
    if span.text[0:7] == "InSight":
        mars_weather = span.text
    else:
        pass

In [13]:
print(mars_weather)

InSight sol 543 (2020-06-06) low -92.9ºC (-135.3ºF) high -6.6ºC (20.2ºF)
winds from the WNW at 7.2 m/s (16.0 mph) gusting to 21.0 m/s (47.0 mph)
pressure at 7.40 hPa


# Grabbing some Mars Facts

In [2]:
url = "https://space-facts.com/mars/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [3]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# df = df[0]
# mars_info_table = df.to_html(index=False)

In [4]:
df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

# Mars Hemispheres!

In [16]:
# Save the base url to add to the img url later
url_base = "https://astrogeology.usgs.gov"

# Create a list of urls to scrape through
url_list = ["https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced", "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced", "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced", "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"]

# Create an empty list to store each image url in
hemisphere_image_urls = []

# Go through the different urls and scrape what is needed. Add to a dictionary and append to the list. 
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Grab the title of the image
    title = soup.find("h2", class_="title").text
    # Create the full image url by combining the url base with the image url
    img = url_base + (soup.find("img", class_="wide-image")["src"])
    # Create a dictionary of the title and the url
    img_dict = {
        "title" : title,
        "image_url" : img
    }
    # Throw that dictionary into a list for parsing later. 
    hemisphere_image_urls.append(img_dict)

In [17]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]